In [1]:
# Parameters
username = "J"


In [2]:
# Importing necessary libraries 
import sys
import numpy as np
import emcee
import astropy.io.fits as pyfits

import matplotlib
matplotlib.use('TkAgg')  # For non-interactive, file-based rendering
import matplotlib.pylab as pl
import random,os
from multiprocessing import Pool
from multiprocessing import cpu_count
import time
from astropy.io import ascii
import corner
from astropy.table import Column
from astropy.table import join





/Users/suddin/.matplotlib is not a writable directory


Matplotlib created a temporary cache directory at /var/folders/gw/w6qdd1bn6zb2zrvd6dz7_9400000gp/T/matplotlib-rext2qe9 because there was an issue with the default path (/Users/suddin/.matplotlib); it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


dyld[53432]: symbol '_CGLSetCurrentContext' missing from root that overrides /System/Library/Frameworks/OpenGL.framework/Versions/A/Libraries/libGL.dylib. Use of that symbol in /System/Library/Frameworks/OpenGL.framework/Versions/A/OpenGL is being set to 0xBAD4007.
dyld[53432]: symbol '_CGLGetCurrentContext' missing from root that overrides /System/Library/Frameworks/OpenGL.framework/Versions/A/Libraries/libGL.dylib. Use of that symbol in /System/Library/Frameworks/OpenGL.framework/Versions/A/OpenGL is being set to 0xBAD4007.
dyld[53432]: symbol '_gll_noop' missing from root that overrides /System/Library/Frameworks/OpenGL.framework/Versions/A/Libraries/libGL.dylib. Use of that symbol in /System/Library/Frameworks/OpenGL.framework/Versions/A/OpenGL is being set to 0xBAD4007.


dyld[53433]: symbol '_CGLSetCurrentContext' missing from root that overrides /System/Library/Frameworks/OpenGL.framework/Versions/A/Libraries/libGL.dylib. Use of that symbol in /System/Library/Frameworks/OpenGL.framework/Versions/A/OpenGL is being set to 0xBAD4007.
dyld[53433]: symbol '_CGLGetCurrentContext' missing from root that overrides /System/Library/Frameworks/OpenGL.framework/Versions/A/Libraries/libGL.dylib. Use of that symbol in /System/Library/Frameworks/OpenGL.framework/Versions/A/OpenGL is being set to 0xBAD4007.
dyld[53433]: symbol '_gll_noop' missing from root that overrides /System/Library/Frameworks/OpenGL.framework/Versions/A/Libraries/libGL.dylib. Use of that symbol in /System/Library/Frameworks/OpenGL.framework/Versions/A/OpenGL is being set to 0xBAD4007.


Matplotlib is building the font cache; this may take a moment.


In [3]:
ncpu = cpu_count()
print("{0} CPUs".format(ncpu))
os.environ["OMP_NUM_THREADS"] = "1"

12 CPUs


In [4]:
np.float = float    
np.int = int   #module 'numpy' has no attribute 'int'
np.object = object    #module 'numpy' has no attribute 'object'
np.bool = bool    #module 'numpy' has no attribute 'bool'

In [5]:
# Parameters
filter = f"{username}"

print(filter)
#data
file = filter+'_sbfj21_update3.csv' # change the filaname here if needed for other bands or for other calibrators
tab = ascii.read('../data/working/'+file)

print(file)

J
J_sbfj21_update3.csv


In [6]:
c = 300000.
q=-0.53

In [7]:
if not 'SN2011iv' in tab['sn']:
    
    #B
    tab.add_row({'sn':'SN2011iv','dist':31.453,'edist':0.084, 'st':0.699,'est':0.007,'Mmax':12.78, 'eMmax': 0.008,'covMs':1.171e-05,'covBV_M':6.4e-05 })
    #H
    #tab.add_row({'sn':'SN2011iv', 'st':0.699,'est':0.007,'Mmax':12.78, 'eMmax': 0.033,'covMs':-3.753e-05,'covBV_M':0.0 })


#if not 'SN20016ajf' in tab['sn']: ## on g-band 
 #       tab.add_row({'sn':'SN2016ajf', 'st':0.493,'est':0.016,'Mmax':16.367, 'eMmax': 0.057, 'covMs':0.0,'covBV_M':0.0 })

    

#tab = ascii.read('../data/working/'+file)
st = tab['st']
est = tab['est']
zhel = tab['zhel']
zcmb = tab['zcmb']
mmax = tab['Mmax']
emmax = tab['eMmax']
bv = tab['BV']
ebv = tab['eBV']
m_csp = tab['m']
eml = (tab['m']-tab['ml'])
emu = (tab['mu']-tab['m'])
em = (emu+eml)/2.
dist = tab['dist']
edist = tab['edist']
c_ms = tab['covMs']
c_mbv = tab['covBV_M']
sn = tab['sn']
ebv_host = tab['EBVmw']
# change SNhunt281

#adding more calibrators from Garnavich 2023
#1
c1 = np.where(sn =='SN2006mr') #31.200 ± 0.093
dist[c1]=31.2
edist[c1]=0.093
#print(sn[c1],dist[c1])
#2
c2 = np.where(sn =='SN2007on') # 31.453 ± 0.084
dist[c2]=31.453
edist[c2]=0.084
#print(sn[c2],dist[c2])
#3
c3 = np.where(sn =='SNhunt281')# 32.369,0.078 same as SN2015bp from Jensen 2021
dist[c3]=32.369
edist[c3]=0.078
#print(sn[c3],dist[c3])

# #4
c4 = np.where(sn =='SN2011iv')# # 31.453 ± 0.084
dist[c4]=31.453
edist[c4]=0.084
# print(sn[c4],dist[c4])
# #4
c5 = np.where(sn =='SN2016ajf')# # 31.453 ± 0.084
dist[c5]=34.202
edist[c5]=0.106
# print(sn[c4],dist[c4])



In [8]:
#Selection

#selection = (tab['dist']<0) & (tab['st']<0.75)  & (tab['EBVmw']<0.15) & (tab['zcmb']>0.01)
#w0 = np.where(selection &  (tab['sn']!='CSP14abk') &  (tab['sn']!='PTF13dyt') &  (tab['sn']!='PTF13dym') & (tab['sn']!='PTF14yw') & (tab['sn']!='PS1-13eao') & (tab['subtype']!='Ia-SC') & (tab['subtype']!='Ia-02cx') & (tab['sn']!='LSQ14fmg')& (tab['sn']!='SN2004dt')& (tab['sn']!='SN2005gj')& (tab['sn']!='SN2005hk')& (tab['sn']!='SN2006bt')& (tab['sn']!='SN2006ot')& (tab['sn']!='SN2007so')& (tab['sn']!='SN2008ae')& (tab['sn']!='SN2008bd')& (tab['sn']!='SN2008ha')& (tab['sn']!='SN2008J')& (tab['sn']!='SN2009dc')& (tab['sn']!='SN2009J')& (tab['sn']!='SN2010ae'))
# no cut
#w0 = np.where((tab['sn']!='CSP14abk') &  (tab['sn']!='PTF13dyt') &  (tab['sn']!='PTF13dym') & (tab['sn']!='PTF14yw') & (tab['sn']!='PS1-13eao') & (tab['subtype']!='Ia-SC') & (tab['subtype']!='Ia-02cx') & (tab['sn']!='LSQ14fmg')& (tab['sn']!='SN2004dt')& (tab['sn']!='SN2005gj')& (tab['sn']!='SN2005hk')& (tab['sn']!='SN2006bt')& (tab['sn']!='SN2006ot')& (tab['sn']!='SN2007so')& (tab['sn']!='SN2008ae')& (tab['sn']!='SN2008bd')& (tab['sn']!='SN2008ha')& (tab['sn']!='SN2008J')& (tab['sn']!='SN2009dc')& (tab['sn']!='SN2009J')& (tab['sn']!='SN2010ae') & (tab['dist']<0))
#w1 = np.where((tab['dist']>0) & (tab['st']<0.75) & (tab['EBVmw']<0.15))
#w1 = np.where(tab['dist']>0)


# Mark's list for B-band 
supernovae = Column([
    "SN2005bl", "SN2006bd", "SN2006gt", "SN2006ob", "SN2007al",
    "SN2007mm", "SN2007ol", "SN2007ux", "SN2008R", "SN2009F",
    "CSP13aao", "KISS15m", "LSQ11pn", "LSQ12fvl", "LSQ13dkp",
    "LSQ14act", "LSQ14ajn", "LSQ14ip", "LSQ14jp", "PTF13ebh",
    "PTF14w", "SN2011jn", "SN2012ij", "SN2013ay"
], name="Supernovae")



w0=[]

supernova_array = np.array(supernovae)
new_array = np.array(sn)
print(len(supernova_array))

# Find indices of matching entries from supernova_column in new_column
match = np.array([np.where(new_array == name)[0][0] if name in new_array else -1 for name in supernova_array])

for name, idx in zip(supernova_array, match):
    if idx != -1:
        #print(f"{name} found at index {idx} in new_column")
        
        w0.append(idx)
    else:
        pass
        #print(f"{name} not found in new_column")
        

### Calibrators B          
calibrators = Column([
    "SN2007cv",
    "SN2008hs",
    "SN2010Y",
    "SN2014bv",
    "SN2016ajf",
    "SN2006mr",
    "SN2007on",
    "SN2008R",
    "SN2011iv",
    "PTF13ebh",
    "CSP15aae",
    "SNhunt281"
],name='Calibrators') 




w1=[]

calibrators_array = np.array(calibrators)
print(len(calibrators_array))

# Find indices of matching entries from supernova_column in new_column
match = np.array([np.where(new_array == name)[0][0] if name in new_array else -1 for name in calibrators_array])

for name, idx in zip(calibrators_array, match):
    if idx != -1:
        #print(f"{name} found at index {idx} in new_column")
        
        w1.append(idx)
    else:
        pass
        #print(f"{name} not found in new_column")
         
        
        
        
        
print (len(st[w0]),len(st[w1]))

#print(ebv[w0])
print(dist[w1])





24
12
19 9
 dist 
------
34.459
33.861
  31.2
31.453
 33.66
31.453
33.296
34.267
32.369


In [9]:
#initial guess
plim=-17, -20
p1lim =-10,20
p2lim = -20,20
rvlim =-5,5
siglim=0,1
h0lim= 50.0,100

In [10]:
# Eqn 9 of Bruns 2018
def distmod(h,z1,z2):
    t1 = (1+z1)/(1+z2)
    t2 = (c*z2)/h
    t3 = 1 + (((1-q)/2)*z2)
    return (5*np.log10(t1*t2*t3)) +25


In [11]:
# Liklihood function
def like(par):
    p,p1,p2,rv,sig,h0 = par

    mu_obs=[]
    mu_model=[]
    mu_stat=[]
    
    if  -25.0<p<14.0  and -100.0<p1<100.0 and -100.0<p2<100.0 and -100.<rv<100.0 and 0<sig<100. and  0< h0 < 1000.0: # priors
        

        
        mu_sn = mmax[w0] - p - p1*(st[w0] - 0.5) -  p2*(st[w0] - 0.5)**2 - rv*(bv[w0]-0.4) #- alpha*(m_csp[w0]-np.median(m_csp[w0]))

        mu_cal = mmax[w1] - p - p1*(st[w1] - 0.5) -  p2*(st[w1] - 0.5)**2 - rv*(bv[w1]-0.4) #- alpha*(m_csp[w1]-np.median(m_csp[w1]))
        
        for mu_sn in mu_sn:
            mu_obs.append(mu_sn)
        for mu_cal in mu_cal:
            mu_obs.append(mu_cal)
        mu_model_sn = distmod(h0,zhel[w0],zcmb[w0])
        mu_model_cal = dist[w1]
        for mu_model_sn in mu_model_sn:
            mu_model.append(mu_model_sn)
        for mu_model_cal in mu_model_cal:
            mu_model.append(mu_model_cal)
        
        fac= (p1+(2*p2*(st[w0]-0.5)))
        fac1= (p1+(2*p2*(st[w1]-0.5)))
        
        vel = 300 ## chening how it effect rersult

        err = (fac*est[w0])**2 +emmax[w0]**2 +(rv*ebv[w0])**2+2*fac*c_ms[w0]+rv*c_mbv[w0]+sig**2 + +(0.00000723*vel/zcmb[w0])**2#+(alpha*em[w0])**2
        err1 = ((fac1*est[w1])**2) +(emmax[w1]**2) +((rv*ebv[w1])**2)+(2*fac1*c_ms[w1])+(rv*c_mbv[w1])+sig**2+(edist[w1]**2)#+(alpha*em[w1])**2

        for err in err:
            mu_stat.append(err)
        for err1 in err1:
            mu_stat.append(err1)
        mu_stat=np.sqrt(mu_stat)

        mu_obs = np.array(mu_obs)
        mu_model = np.array(mu_model)
        mu_stat = np.array(mu_stat)
        dmu=mu_obs-mu_model
        
        chi =np.sum((dmu)**2/mu_stat**2)
        return -0.5*chi - (0.5*np.sum(np.log(2*np.pi*(mu_stat)**2))) 
    else:
        return -np.inf

In [12]:
# EMCEE
ndim, nwalkers = 6, 60
ssize=3000
burnin = 1000


p00 = np.random.rand(nwalkers) * (plim[1] - plim[0]) + plim[0]
p10 = np.random.rand(nwalkers) * (p1lim[1] - p1lim[0]) + p1lim[0]
p20 = np.random.rand(nwalkers) * (p2lim[1] - p2lim[0]) + p2lim[0]
rv0 = np.random.rand(nwalkers) * (rvlim[1] - rvlim[0]) + rvlim[0]
sig0 = np.random.rand(nwalkers) * (siglim[1] - siglim[0]) + siglim[0]
h00 = np.random.rand(nwalkers) * (h0lim[1] - h0lim[0]) + h0lim[0]

p0 = np.array([p00,p10,p20,rv0,sig0,h00]).T


In [13]:
sampler = emcee.EnsembleSampler(nwalkers, ndim, like)
print ("running mcmc on "+file)
start = time.time()
sampler.run_mcmc(p0,ssize,progress=True)
samples = sampler.chain[:, burnin:, :].reshape((-1, ndim))
end = time.time()
serial_time = end - start
print ("Mean acceptance fraction:", np.mean(sampler.acceptance_fraction))

running mcmc on J_sbfj21_update3.csv


  0%|          | 0/3000 [00:00<?, ?it/s]

dyld[53440]: symbol '_CGLSetCurrentContext' missing from root that overrides /System/Library/Frameworks/OpenGL.framework/Versions/A/Libraries/libGL.dylib. Use of that symbol in /System/Library/Frameworks/OpenGL.framework/Versions/A/OpenGL is being set to 0xBAD4007.
dyld[53440]: symbol '_CGLGetCurrentContext' missing from root that overrides /System/Library/Frameworks/OpenGL.framework/Versions/A/Libraries/libGL.dylib. Use of that symbol in /System/Library/Frameworks/OpenGL.framework/Versions/A/OpenGL is being set to 0xBAD4007.
dyld[53440]: symbol '_gll_noop' missing from root that overrides /System/Library/Frameworks/OpenGL.framework/Versions/A/Libraries/libGL.dylib. Use of that symbol in /System/Library/Frameworks/OpenGL.framework/Versions/A/OpenGL is being set to 0xBAD4007.
  0%|          | 1/3000 [00:00<07:54,  6.32it/s]

  0%|          | 2/3000 [00:00<07:15,  6.89it/s]

  0%|          | 4/3000 [00:00<06:13,  8.01it/s]

...

100%|█████████▉| 2995/3000 [04:10<00:00, 12.45it/s]

100%|█████████▉| 2997/3000 [04:10<00:00, 12.29it/s]

100%|█████████▉| 2999/3000 [04:10<00:00, 12.34it/s]

100%|██████████| 3000/3000 [04:10<00:00, 11.96it/s]

Mean acceptance fraction: 0.5162


In [14]:
# Chains
fig, axes = pl.subplots(6, figsize=(10, 7), sharex=True)
samples = sampler.get_chain()
labels = ["$M_B$","P1", "P2", r"$\beta$", r"$\sigma_{int}$", r"$H_0$"]
for j in range(ndim):
    ax = axes[j]
    ax.plot(samples[:, :, j], "k", alpha=0.3)
    ax.set_xlim(0, len(samples))
    ax.set_ylabel(labels[j])
    ax.yaxis.set_label_coords(-0.1, 0.5)

axes[-1].set_xlabel("step number")

fig.savefig("../results/trippmethod/"+file[:-4]+"_chains.pdf")
#pl.show()

In [15]:
samples = sampler.chain[:, burnin:, :].reshape((-1, ndim))
#tau = sampler.get_autocorr_time()
#print(tau)

 # Printing results
for i, name in enumerate(labels):
    mcmc = np.percentile(samples[:, i], [16, 50, 84])
    q = np.diff(mcmc)
    print(f"{name}: {mcmc[1]:.3f} (+{q[1]:.3f}/-{q[0]:.3f})")
    
    

$M_B$: -18.034 (+0.066/-0.066)
P1: -2.484 (+1.041/-1.062)
P2: 7.552 (+2.723/-2.669)
$\beta$: 0.591 (+0.443/-0.455)
$\sigma_{int}$: 0.104 (+0.032/-0.027)
$H_0$: 74.751 (+2.204/-2.066)


In [16]:
# Writing Result
p0_mcmc,p1_mcmc,p2_mcmc,beta_mcmc,sig_mcmc, H0_mcmc = map(lambda v: (v[1], v[2]-v[1], v[1]-v[0]),
                             zip(*np.percentile(samples, [16, 50, 84],
                                                axis=0)))



f1 =open('../results/trippmethod/'+file[:-4]+'_result_tripp.txt','w') ### MUST change !!

f1.write('%s\t%s\t%s\t%s\t%s\t%s\n'%('p0','p1','p2','beta','sig_int','H0'))



f1.write('%.3f\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f\n'%(p0_mcmc[0],p1_mcmc[0],p2_mcmc[0],beta_mcmc[0],sig_mcmc[0],H0_mcmc[0]))

f1.write('%.3f\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f\n'%(p0_mcmc[1],p1_mcmc[1],p2_mcmc[1],beta_mcmc[1],sig_mcmc[1],H0_mcmc[1]))
f1.write('%.3f\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f\n'%(p0_mcmc[2],p1_mcmc[2],p2_mcmc[2],beta_mcmc[2],sig_mcmc[2],H0_mcmc[2]))

f1.close()

In [17]:
# Triangle plot
figure = corner.corner(samples, labels = ["$M_B$","P1", "P2",r"$\beta$", r"$\sigma_{int}$", r"$H_0$"])
figure.savefig("../results/trippmethod/"+file[:-4]+"_tripp"+str(nwalkers)+"_"+str(ssize)+".pdf")


In [18]:
#pl.show()

In [19]:
from IPython.display import clear_output

clear_output(wait=True)